    1. AllContacts 전처리 (testID 삭제) 
    2. AllContacts, AccountDB 매핑
    3. 파생변수 생성: 직급, 산업, isfam, ib/ob, 고객등급, Mapping
    4. Grouping
    5. ContactDB 생성 

In [1]:
import pandas as pd 
import numpy as np 
import os

import warnings 
warnings.filterwarnings(action='ignore') 

data_path =  '../../data/raw_data/'
accountDB = pd.read_excel(data_path + 'Account_2022-07-06.xlsx')


In [2]:
# allcontacts 안에 test 데이터 삭제, 필요없는 변수 삭제 
def delete_testID(contactDB, delete_contacts, del_colnames):
   
    # CNS 에 소속된 고객 삭제 
    cns_name = ['LG CNS','LG_CNS', 'LGCNS', 'Lgcns', 'lgcns', 'cns', 'MS CLOUD 사업팀 | LG CNS', '솔루션영업팀 / lgcns']
    index_list = []
    for  i in range(len(cns_name)):
        cns = cns_name[i]
        idx = list(contactDB[contactDB.Company == cns].index)
        index_list += idx
    contactDB.drop(index_list, axis = 0, inplace = True)
   
    cns = contactDB[contactDB.Company_Mapping_Code == 'lgcnscom'].index
    contactDB.drop(cns, inplace = True)

    # test_contactId.txt 파일에 있는 ContactID 삭제 
    delete_contacts = pd.Series(list(delete_contacts.iloc[0,:])).dropna()
    delete_contacts = [x for x in delete_contacts if np.isnan(x) == False]# txt 파일 변환 과정에서 생긴 nan값 삭제
    contactID = list(contactDB.ContactID)
    mask = ~contactDB.ContactID.isin(delete_contacts)
    df = contactDB[mask]
    
    df.drop(labels = del_colnames, axis = 1, inplace=True)# 미사용 변수 삭제 
    return df.reset_index(drop=True)


In [3]:
# 매핑 불가한 contact 우선 제쳐두기 (cannot map)
def mapping0(contactDB):
    personal = ['naver.com', 'gmail.com', 'hanmail.net', 'daum.net', 'nate.com', 'hotmail.com', 'kakao.com', 'yahoo.com', 'Yahoo.com',\
                'icloud.com' 'paran.com', 'empal.com' ,'yahoo.co.kr', 'outlook.com']
    contact_email = list(contactDB['Email Address Domain'])

    index = []
    for i in range(len(contact_email)):
        email = contact_email[i]
        if email in personal:
            index.append(i)
        else:
            index.append(None)
    index = pd.Series(index).dropna()
    temp = contactDB.iloc[index, ]

    cannot_map = temp[(temp.Company.isna()) & (temp.Company_Mapping_Code.isna())]

    cannot_map['CompanyID'] = None
    to_map = contactDB.drop(cannot_map.index) # 매핑 해야하는 contact
    print('mapping0 결과 -> to_map.shape: ', to_map.shape, 'cannot_map.shape: ', cannot_map.shape)
    return cannot_map, to_map

In [4]:
# Mapping
def mapping_code_id(mapping_code):
    global accountDB
    if mapping_code in list(accountDB.Company_Mapping_Code):
        idx = list(accountDB.Company_Mapping_Code).index(mapping_code)
        return accountDB['CompanyID'][idx]
    else:
        return None

def use_email(email, email_list):
    if email in email_list:
        idx = email_list.index(email)
        companyid = accountDB['CompanyID'][idx]
        return companyid 
    else:
        return None
    
def manipulate_string(string): # 문자열에서 띄워쓰기 없애고 소문자로 바꾸는 함수 
    string = str(string)
    res = string.strip()
    return res.lower()

def get_companyID(Email): # 이메일 사용
    if Email == 'hanafn.com':
        companyId = 1150 #하나금융지주 
    elif Email == 'kolon.com':
        companyId = 1390 #코오롱 
    elif Email == 'shinhan.com':
        companyId = 1641 # 신한금융지주 
    else:
        companyId = None 
    return companyId

def get_companyID2():
    company = ['(주)두산','두산'] + ['롯데쇼핑(주)롯데마트롭스사업본부','롯데쇼핑(주)마트사업본부'] \
    + ['현대자동차 미국법인 (재경팀)','씨티은행','삼성', 'Samsung','Hansem, Inc.', '한샘이펙스','Skt',
       '부산시','농협생명보험', '농협손해보험','효성티앤에스','한국산업기술평가관리원','귀뚜라미',
       '육군협회','삼성산업','Konkuk University','동덕여자대학교 학생생활연구소', '동덕여자대학교학생생활연구소',
       '서울과기대','한국전력기술','한국가스기술공사','정보통신','제주특별자치도','고려대',
       '다임','대상그룹','티앤씨','동진','한국조선해양','단국대학교의과대학','한국컴퓨터(주)', 
    '우리금융지주','동원무역','제주대하쿄','KT/융합ICT1TF','국가정보자원관리원','INITECH', '이니텍(주)']

    companyid = [1061]*2 + [1483]*2 + [1537,3380] + [1613]*2 + [1173]*2 + [1683,2651]+[2022]*2 \
    +[1243,1809,2395,3080,2695,1397]+[1053]*2 + [1802,1342,1388,3393,3154,1765,1793 ,1020,1246,
                                                  2528,3395,1026,1998,3022,1059,1299,1428,1948]+[3088]*2
    category = pd.DataFrame({'Company':company,'CompanyID':companyid})
    return category

def get_companyID3(Company): # SK 그룹
    if pd.isna(Company):
        return 1664 #sk그룹 
    elif Company in ['SK 그룹, 에스케이 그룹',  'FSK L&S',  'SKTELINK', 'Sk 넥실리스', 'Sk 바이오사이언스', 'SK하이닉스', 'SKT']:
        return 1664 #sk그룹
    elif ('CC' in str(Company)) or ('C&C' in str(Company)) or ('cloud / SK C&C'in str(Company)):
        return 1665 # SK C&C
    elif '십일번가주식회사' in str(Company):
        return 1663 #11번가
    elif  Company =='SKshieldus':
        return 1792
    elif Company in ['SK Networks Service', 'SKNS']:
        return 1668 # SK네트웍스서비스
    elif Company in ['ES', 'SK ES']:
        return 1666 # SK E&S
    elif Company == 'SK On':
        return 1673 # sk이노베이션
    else:
        return 1664


# 1. ContactDB기준 Company Mapping Code가 NA가 아닌 값 우선
def mapping1(to_map):
    mapped1 = to_map[to_map.Company_Mapping_Code.notna()]
    idx = mapped1.index
    mapping_code = list(mapped1.Company_Mapping_Code)
    companyid = list(map(mapping_code_id, mapping_code)) # mapping_code_id 함수사용
    mapped1['CompanyID'] = companyid 
    print('mapping1 결과 -> mapped1.shape: ', mapped1.shape)
    return mapped1


# 2. ContactDB기준 Company Mapping Code가 NA인 값 
def mapping2(to_map, accountDB):
    to_map = to_map[to_map.Company_Mapping_Code.isna()]
    notna = to_map[to_map.Company_Mapping_Code.notna()]
    
    email_list = list(accountDB[accountDB.is_identify_by_email == 'Y']['회사 이메일 도메인'])
    email_domain = list(to_map['Email Address Domain'])

    companyid = []
    for i in range(len(email_domain)):
        email = email_domain[i]
        c_id = use_email(email, email_list) # use_email 함수 사용 
        companyid.append(c_id)

    to_map['CompanyID'] = companyid
    mapped2= to_map[to_map.CompanyID.notna()]
    to_map = to_map[to_map.CompanyID.isna()]
    
    print('mapping2 결과 -> to_map.shape: ', to_map.shape, 'mapped2.shape: ', mapped2.shape)
    return to_map, mapped2


# 3. 이름 확인 
def mapping3(to_map, accountDB):
    to_map.reset_index(drop = True, inplace=True)

    not_mapped_company = list(map(manipulate_string, list(to_map.Company))) # manipulate_string 함수 사용
    accountDB_company = list(map(manipulate_string, list(accountDB['Company Name'])))

    contact_index = []; companyID = []
    for i in range(len(not_mapped_company)):
        company = not_mapped_company[i]
        if company in accountDB_company:
            account_idx = accountDB_company.index(company)
            companyid = accountDB['CompanyID'][account_idx]
            companyID.append(companyid)
            contact_index.append(i)
        else:
            pass

    mapped3 = to_map.iloc[contact_index]
    mapped3['CompanyID'] = companyID
    to_map.drop(contact_index, axis=0, inplace = True) # 미분류 
    print('mapping3 결과 -> to_map.shape: ', to_map.shape, 'mapped3.shape: ', mapped3.shape)
    return to_map, mapped3

# 자동화 어려운 회사명은 직접 확인 후 매핑 

def mapping4(to_map):
    contact_email = list(to_map['Email Address Domain'])
    companyId = list(map(get_companyID, contact_email))
    to_map['CompanyID'] = companyId 
    mapped4 = to_map[to_map.CompanyID.notna()]
    to_map = to_map.drop(mapped4.CompanyID)
    print('mapping4 결과 -> to_map.shape: ', to_map.shape, 'mapped4.shape: ', mapped4.shape)
    return to_map, mapped4

def mapping5(to_map):
    category = get_companyID2()
    temp = pd.merge(to_map.drop(['CompanyID'],axis=1), category, on='Company',how='left')
    mapped5 = temp[temp.CompanyID.notna()]
    to_map =  temp[temp.CompanyID.isna()]
    print('mapping5 결과 -> to_map.shape: ', to_map.shape, 'mapped5.shape: ', mapped5.shape)
    return to_map, mapped5

def mapping6(to_map, cannot_map):
    to_map.reset_index(drop = True, inplace=True)
    # 매핑 코드, 등등 다 걸어준 후에 돌리는 함수 
    to_delete = ['.','-','무','1','대학생','회사','a', 'd', 'dd', '..', 'x', '개인', 'free', '집', \
                 '개인사업', '개인사업자', 'none', 'X', 'IT', 'ABC', \
                 'ㅇ', '비공개', 'w', '개인사업', '-', 'd', '--', '대학교', 'j',\
                 'dps','ㆍ', '백수', '...','it', '123' ,'ㅇㅇ','초등학교','abc','aaa',\
                 '유진','대학원생','ss','병원','군', 'None', '11','일반인','*','회사명',\
                 'aa','연구소','asdfasdf','취준','무직','농업','없음','ㅁㄴㅇㄹ','n','ㄴ',\
                 '111','강사', '대학원', 'ㅡ','fd','미정','df', '김','유라','서연','가나다','김만기'\
                 '정윤상','정윤상','aaabbcc','(개인)', '직장인','서울','자','휴직','kk','신백초등학교',\
                 '대학','G', 'No','Freelancer','.','Home', '김만기','Freelance', None]

    contact_company = list(to_map.Company)
    index = []
    for i in range(len(contact_company)):
        company = contact_company[i]
        if company in to_delete:
            index.append(i)
        else:
            index.append(None)

    idx = list(pd.Series(index).dropna())
    temp = to_map.iloc[idx, :]
    temp['CompanyID'] = None
    cannot_map = cannot_map.append(temp)
    to_map = to_map.drop(idx)  # to_map, cannot_map, not_mapped 
    not_mapped = to_map.copy()
    not_mapped.reset_index(drop = True, inplace=True)
    
    #sk그룹만 
    sk_idx = list(not_mapped[not_mapped['Email Address Domain'] =='sk.com'].index)
    sk_to_map = not_mapped.iloc[sk_idx,:]
    not_mapped = not_mapped.drop(sk_idx)
    CompanyID = list(map(get_companyID3, list(sk_to_map.Company)))
    sk_to_map['CompanyID'] = CompanyID
    
    mapped6 = sk_to_map.copy() #mapped6, not_mapped

    print('mapping6 결과 \n-> cannot_map.shape',cannot_map.shape, 'to_map.shape: ', to_map.shape, 'mapped6.shape: ', mapped6.shape)
    return cannot_map,not_mapped, to_map, mapped6

In [5]:
def Map_Contact_Account(contactDB, accountDB):
    # Mapping
    cannot_map, to_map = mapping0(contactDB)
    mapped1 = mapping1(to_map)
    to_map, mapped2 = mapping2(to_map, accountDB)
    to_map, mapped3 = mapping3(to_map, accountDB)
    to_map, mapped4 = mapping4(to_map)
    to_map, mapped5 = mapping5(to_map)
    cannot_map, not_mapped, to_map, mapped6 = mapping6(to_map, cannot_map)

    mapped = pd.concat([mapped1, mapped2, mapped3, mapped4, mapped5, mapped6])
    mapped['Mapping'] = 1
    not_mapped['CompanyID'] = None
    not_mapped['Mapping'] = 0
    cannot_map['CompanyID'] = None
    cannot_map['Mapping'] = 0

    ContactDB2 = pd.concat([mapped, cannot_map, not_mapped]).drop_duplicates()
    
    Map_Account = accountDB.loc[:,['CompanyID', 'Company Name', '기업등급(5년평균)', '산업유형', '산업상세']]
    Map_Account.columns = ['CompanyID', 'Company', '고객등급', '산업유형', '산업상세']
    mapped_c = ContactDB2[ContactDB2.Mapping == 1].drop('Company', axis = 1)
    mapped_c = pd.merge(mapped_c, Map_Account, how = 'left', on ='CompanyID')

    notmapped_c = ContactDB2[ContactDB2.Mapping == 0]
    notmapped_c['고객등급'] = 'unknown'

    ContactDB = pd.concat([mapped_c, notmapped_c])
    return ContactDB

In [6]:
# 직급변수 통일 
# Title변수의 값이 결측치인 관측치에 대해 다른 직급 변수를 근거로 imputation 
# col = '직급'
def get_title1(title):
    jsm = ['시니어컨설턴트', '선임', '담당', '과장', '차장', '대리', '부부장', '계장', '실장', '매니저', '사원', 'PM', 'SeniorSpeciallist', '시니어매니저', '시니어미내저',
    '연구원', '책임연구원', '전임', '주임', '담당자', '선임연구원', '사원 ', '책임 ', 'manager', '비상안전기획관', '부실장',
    'Business Develop manager for Finance Industry Service', '분부장','정보보호진단분석담당', 'TM', 'Manager', 'Project Manager']
    tl = ['파트장', '팀장', '팀장/전문위원', '개발리더(연구위원)', '팀장(전임)', 'TF장', '센터장', '수석', '지점장', '부장', '부문장', "팀장\u3000", '팀원', '부장 ', '위원',
    '팀장(전문위원)', '연구위원', 'Task Leader', 'TP Leader', '팀장 ', '연구위원/TP Leader', '연구위원/Task Leader', '전문위원', 'Business Leader', '그룹장', '부서장',
    '전문위원 ', '대표/소장', '팀장/부국장', 'Principal Consultant ', '시니어매니저/팀장', 'head', '수석팀장']
    ex = ['상무', '수석연구위원(전무)', '수석연구위원(상무)', '수석전문위원(상무)', '이사', 'CIO', '본부장', '상무보', '고문', '상임이사', '부문장(상무)', '임원', 'COO/PO', 'CTO',
    'ciso', '본부장/CIO']
    ve = ['부사장', '수석연구위원(부사장)', '부회장', 'C-Level / VP / Director']
    ceo = ['대표', '회장', 'CEO', 'CEO/CIO', '대표이사', '연구소장/대표이사', 'President']

    if title in jsm:
        return 'Junior/Senior Manager'
    elif title in tl:
        return 'Team Leader'
    elif title in ex:
        return 'Executive'
    elif title in ve:
        return 'Vice-Executive'
    elif title in ceo:
        return 'CEO'
    else:
        return 'Unknown'

def get_title2(title):
    prof = ['Adjunct Professor', 'Professor', '교수', '교수진', '조교수', '연구교수']
    jsm2 = ['사원 ', '임직원', '기장', '선임 유틸리티', '과장', '매니저겸 총무', '매니저', '파트장', '사원', '대리', '생산직', 'PM', '직원', '차장', '책임', '선임','수석', '담당',
    '주임', '마케팅 실장', '토목', '연구원', '선임연구원', '실장', '장애인 복지 ', 'Unit Manager', '영양사','부부장','책임 연구원', '반장', 'Solution Engineer / 과장']
    useless = ['Attorney', 'Public Accountant', '인턴', '근로자', '교직원', '상담원', '조교', '시설관리', '종업원', '학부생', '학생', '프리랜서강사', '직급이 없습니다. ',
    '무직', '교사', '학생 고 3', '대학원생', '대학생', 'cadet', '교육생', '강사', 'X', '주부', '없음']
    unkn = ['.', '-']
    tl2 = ['부장', '지점장', '전문위원', '부서장']
    ex2 = ['고문', '자문역', '부문장 ', '부문장', '상근자문']
    if title in prof:
        return 'Professor'
    elif title in jsm2:
        return 'Junior/Senior Manager'
    elif title in useless:
        return 'useless'
    elif title in tl2:
        return 'Team Leader'
    elif title in ex2:
        return 'Executive'
    else:
        return 'Unknown'

def unify_title_columns(ContactDB):
    ContactDB['직급'] = pd.Series(list(map(get_title1, list(ContactDB['직급']))))
    ContactDB['Title (Others)'] = pd.Series(list(map(get_title2, list(ContactDB['Title (Others)']))))
    
    for i in range(len(ContactDB)):
        if ContactDB.Title.isna().iloc[i] == True:
            ContactDB.Title.iloc[i] = ContactDB['직급'].iloc[i]
        elif ContactDB.Title.iloc[i] == 'Others':
            ContactDB.Title.iloc[i] = ContactDB['Title (Others)'].iloc[i]
    ContactDB.Title.fillna('Unknown', inplace = True)
    return ContactDB

In [13]:
#Industry 
# AccountDB의 industry 선반영
# 이후 결측치로 남아있는 관측치는 ContactDB의 값으로 대체

def unify_industry_columns(contact, ContactDB):
    contact.loc[:, 'Industry'] = contact.loc[:, '산업유형']
    contact.loc[contact['산업유형'] == '금융;비영리', 'Industry'] = '금융'
    contact.loc[contact['산업유형'] == '기업집단;유통', 'Industry'] = '기업집단'
    contact.loc[contact['산업유형'] == '통신', 'Industry'] = '방송통신'
    contact.loc[contact['산업유형'] == '서비스;비영리', 'Industry'] = '비영리'
    contact.loc[contact['산업유형'] == '비영리', 'Industry'] = '비영리'

    for i in range(len(contact)):
        if contact.Industry.iloc[i] == '서비스':
            if 'IT' in contact.산업상세.iloc[i]:
                contact.Industry.iloc[i] = 'IT서비스'
            else :
                contact.Industry.iloc[i] = '일반서비스'
    for i in range(len(contact)):
        if contact.Industry.isna().iloc[i] == True:
            contact.Industry.iloc[i] = ContactDB.Industry.iloc[i]

    contact.loc[contact['Industry'] == 'Telecommunications & Media', 'Industry'] = '방송통신'
    contact.loc[contact['Industry'] == 'Manufacturing', 'Industry'] = '제조'
    contact.loc[contact['Industry'] == 'Government & Public', 'Industry'] = '공공'
    contact.loc[contact['Industry'] == 'Govermment & Public', 'Industry'] = '공공'
    contact.loc[contact['Industry'] == 'Financial Services', 'Industry'] = '금융'
    contact.loc[contact['Industry'] == 'Logistics', 'Industry'] = '물류'
    contact.loc[contact['Industry'] == 'IT Service', 'Industry'] = 'IT서비스'
    contact.loc[contact['Industry'] == 'Education', 'Industry'] = '교육'
    contact.loc[contact['Industry'] == 'Transportation', 'Industry'] = '운송'
    return contact

In [8]:
# 계열사여부 
def get_isfam(Company):
    LGgroup = ['lg', '엘지']
    if pd.notna(Company):
        Company = str(Company).lower()
        for i in LGgroup:
            if i in Company:
                return 1
            else:
                return 0
    else:
        return 0

def isfam(contact):
    company = list(contact.Company)
    isfam = list(map(get_isfam, company))
    contact['isfam'] = isfam
    return contact

In [16]:
# in/out (유입경로))
def get_first_LeadSource(LeadHistory):
    if pd.notna(LeadHistory):
        return LeadHistory.split(';')[0]
    else:
        return None

def FirstLead(contact):
    contact.drop(['Lead Source'], axis = 1, inplace = True)
    contact['FirstLead'] = list(map(get_first_LeadSource, list(contact['Lead Source_History2'])))
    return contact

def get_inout(firstlead):
    ib = ['Inquiry', 'BroDN', 'Inquiry_Launchcenter', 'Inquiry_OTSecuTech_2201', 'Letter_newsletter_inquiry', 
     'Inquiry_CNSwithyou']
    
    unrelated = [ 'MAT_Adopt_related']
    
    if pd.isna(firstlead):
        return None
    elif firstlead in ib:
        return 'ib'
    elif firstlead in unrelated:
        return 'ob' 
    else:
        return 'ob'

def first_action(contact, activity_path):
    contact = FirstLead(contact) # FirstLead 함수 사용 
    activity = pd.read_csv(activity_path +'activity_merged.csv')
    first_action = activity.groupby('ContactID').nth(0)[['ActivityType', 'cns_action (major)', 'cns_action (minor)']]
    activity_type = list(first_action.ActivityType)
    cns_action = list(first_action['cns_action (major)'])
    
    first_action['first_action'] = 0
    ib_idx = list(first_action[first_action['cns_action (major)'] == 'homepage'].index)
    first_action['first_action'][ib_idx] = 'ib'

    ob_idx = list(first_action[(first_action['first_action'] != 'ib') & (first_action['cns_action (major)'].notna())].index)
    first_action['first_action'][ob_idx]= 'ob'

    idx = list(first_action.index)
    df = pd.DataFrame({'ContactID': list(first_action.index), 'first_action': list(first_action.first_action)})
    merged = pd.merge(contact,df, how='left', on='ContactID')
    merged = merged[['ContactID', 'CompanyID', 'Title', 'Email Address Domain',\
                     'Title (Others)', 'Industry (KR)', 'Industry', 'isKeyPerson', '직급',\
                     'Lead Source_History2', 'Mapping', 'Company', '고객등급', '산업유형', '산업상세',\
                     'isfam', 'FirstLead', 'first_action']]

    
    temp_df = merged[(merged['first_action'].isna()) | (merged['first_action'] == 0)]
    to_map = merged.drop(list(temp_df.index))
    
    temp_df['first_action'] = list(map(get_inout, list(temp_df.FirstLead)))
    df = pd.concat([temp_df, to_map])
    df.first_action = df.first_action.fillna('ob')
    return df

In [17]:
# Grouping 
def group_title(title):# 직급 변수 
    title_dict = {'C-Level':['Vice-Executive','Executive','CEO'],
                  'Team Leader': ['Team Leader'],
                  'Manager':['Junior/Senior Manager']}

    labels =  list(title_dict.keys())
    if title in title_dict[labels[0]]:
        return labels[0]
    elif title in title_dict[labels[1]]:
        return labels[1]
    elif title in title_dict[labels[2]]:
        return labels[2]
    elif pd.isna(title):
        return 'unknown'
    elif title == 'Unknown':
        return 'unknown'
    else:
        return 'etc'
    
# 산업 변수 
def group_industry(industry):
    if pd.isna(industry):
        return '산업_unknown'
    elif industry == '제조':
        return '산업_제조'
    elif industry == 'IT서비스':
        return '산업_IT서비스'
    elif industry == '일반서비스':
        return '산업_일반서비스'
    elif industry == '금융':
        return '산업_금융'
    else:
        return '산업_others'
    

In [23]:
data_path =  '../../data/raw_data/'
save_path = '../../data/processed1/'
activity_path = '../../data/processed1/'

def run_codes(data_path, activity_path):
    allcontacts = pd.read_excel(data_path + 'All_contacts_download_2022-07-06.xlsx')
    accountDB = pd.read_excel(data_path + 'Account_2022-07-06.xlsx')
    
    print('delete_testID 함수 실행')
    # 미사용 변수 리스트
    del_colnames = ['Date Created', 'Date Modified', 'Eloqua Contact ID', 'Title (KR)', 'Subscribe', 'Department', 'Subscribe Date', 'Interests (Others)',
    '개인정보 수집•이용 동의 (필수)', '개인정보 수집•이용 동의 (선택)', '(필수) 개인정보 수집•이용 동의 날짜', '(선택) 개인정보 수집•이용 동의 날짜', 'Unsubscribe Date', '회사홈페이지',
    'Subscribe Date_Display', 'Unsubscribe Date_Display', '(임시) NCD웨비나참여', '(홈페이지) BlogNews_Subscribe', '(홈페이지) Job Group', 'Interests (KR)',
    'KeyPerson Source', 'KeyPerson Source_History', '(임시) 보안웨비나참여', '(홈페이지) BlogNews_Subscribe Date','(홈페이지) BroDN_Pagename',
    '(홈페이지) BlogNews_Unsubscribe Date', '(홈페이지) BlogNews_Unsubscribe Date_Display', 'Unsubscribe Reason', 'Unsubscribe Reason (Others)',
    'Subscribe (Program)', 'Text Email only', 'withyou_webinarname', 'Contact_Management', 'withyou_eventname', 'Lead Source Code(변환용)']
    delete_contacts = pd.read_csv('./labelling_category/test_contactId.txt', sep = ',', header = None)
    #print('-test계정, 변수 삭제 전 data size:', allcontacts.shape)
    contactDB = delete_testID(allcontacts, delete_contacts, del_colnames)
    #print('-test계정, 변수 삭제 후 data size:', contactDB.shape)

    print('\nMap_Contact_Account함수 실행 -> ContactDB와 AccountDB 매핑')
    contact = Map_Contact_Account(contactDB, accountDB)

    print('\n파생변수 생성')
    contact = unify_title_columns(contact); print('\n-직급변수 done')
    ContactDB = contact
    contact = unify_industry_columns(contact,ContactDB); print('\n-산업변수 done')
    contact = isfam(contact); print('\n-계열사여부 done')
    contact = first_action(contact, activity_path); print('\n-자발비자발 done')
    
    print('\nGrouping')
    contact.Title = contact.Title.apply(group_title);print('\n-직급변수 done')
    contact.Industry = contact.Industry.apply(group_industry);print('\n-산업변수 Done')
    
    
    contact['고객등급'].replace({'Grape':8, 'Pineapple':7, 'Pear':6, 'Mango':4, 'Strawberry':3, 'Banana':2,
       'Apple':1, 'Lemon':5}, inplace = True)
    print('\n-고객등급 Done')
    
    use_col = ['ContactID','CompanyID', 'Title', 'Industry','isfam','first_action', 'Mapping', '고객등급']
    contactDB = contact[use_col]
    print('\nContactDB shape: ', contactDB.shape)
    
    return contactDB 

In [24]:
contact = run_codes(data_path, activity_path)

delete_testID 함수 실행
-test계정, 변수 삭제 전 data size: (36680, 47)
-test계정, 변수 삭제 후 data size: (29437, 12)

Map_Contact_Account함수 실행 -> ContactDB와 AccountDB 매핑
mapping0 결과 -> to_map.shape:  (26976, 12) cannot_map.shape:  (2461, 13)
mapping1 결과 -> mapped1.shape:  (13447, 13)
mapping2 결과 -> to_map.shape:  (13502, 13) mapped2.shape:  (27, 13)
mapping3 결과 -> to_map.shape:  (12844, 13) mapped3.shape:  (658, 13)
mapping4 결과 -> to_map.shape:  (12841, 13) mapped4.shape:  (23, 13)
mapping5 결과 -> to_map.shape:  (12758, 13) mapped5.shape:  (83, 13)
mapping6 결과 
-> cannot_map.shape (2901, 13) to_map.shape:  (12318, 13) mapped6.shape:  (41, 13)

파생변수 생성

-직급변수 done

-산업변수 done

-계열사여부 done

-자발비자발 done

Grouping

-직급변수 done

-산업변수 Done

-고객등급 Done

ContactDB shape:  (29457, 8)


In [25]:
# 확인 
print('Title:')
display(contact['Title'].value_counts(dropna=False))

print('\nIndustry:')
display(contact.Industry.value_counts(dropna=False))

print('\nisfam:')
display(contact.isfam.value_counts(dropna=False))

print('\nfirst_action:')
display(contact.first_action.value_counts(dropna=False))

print('\nMapping:')
contact['Mapping'].value_counts(dropna=False)

print('\n고객등급:')
contact['고객등급'].value_counts(dropna=False)

Title:


Manager        10837
unknown        10559
C-Level         4666
Team Leader     3365
etc               30
Name: Title, dtype: int64


Industry:


산업_unknown    15860
산업_제조          4088
산업_others      3574
산업_IT서비스       2443
산업_금융          1761
산업_일반서비스       1731
Name: Industry, dtype: int64


isfam:


0    26433
1     3024
Name: isfam, dtype: int64


first_action:


ob    24662
ib     4795
Name: first_action, dtype: int64


Mapping:

고객등급:


unknown    15178
5           5889
1           4117
7           1879
3            912
6            726
4            470
2            286
Name: 고객등급, dtype: int64

In [26]:
contact.head()

,ContactID,CompanyID,Title,Industry,isfam,first_action,Mapping,고객등급
5,18,1687.0,C-Level,산업_일반서비스,0,ob,1,5
6,24,1765.0,etc,산업_others,0,ob,1,7
8,39,1451.0,Manager,산업_일반서비스,1,ob,1,5
9,45,1456.0,unknown,산업_제조,1,ob,1,1
10,58,1705.0,Team Leader,산업_일반서비스,0,ob,1,5


In [27]:
save_path = '../../data/processed1/'
contact.to_csv(save_path + 'ContactDB.csv', index = False)